# Data to Pandas table

**Purpose of script:**

Put mw and opt data to one pandas dataframe, create features. One row per pixel.

- In: opt and mw data, all files
- Out: one file (maybe several due to size constraints) (csv or parquet) with table of features prepared to be used in model

In [1]:
import xarray
import pandas as pd
from datetime import datetime
import numpy as np
from tqdm import tqdm

from os import listdir
from os.path import isfile, join

## Functions

In [3]:
def raster_to_pandas(data, mw_or_opt):
    # convert mw to pandas
    data = data.to_dataframe()
    # fix index
    data = data.reset_index()
    # remove columns: spacial_ref, band
    data = data[['x', 'y', 'band_data']]
    # rename
    if mw_or_opt in (['mw', 'microwave']):
        data.rename({'band_data': 'mw_value'}, axis=1, inplace=True)
    elif mw_or_opt in (['opt', 'optical']):
        data.rename({'band_data': 'opt_value'}, axis=1, inplace=True)
        # fill na for masked opt data, don't remember why needed
        data['opt_value'].fillna(-1, inplace=True)
    else:
        print('mw or opt?')
    return data

*Baptiste's :*
- open_mfdataset() 
- concat dim time (to put to ine file?)

In [4]:
def get_neighbors(mat, a, b):
    neighbors = [mat[i][j] if (i > -1 and j > -1 and j < len(mat[0]) and i < len(mat)) else np.nan for i in range(a-1, a+2) for j in range(b-1, b+2) ]
    return neighbors

In [5]:
def get_neighbours_df(data):
    index_list = [(i,j) for i in range(data.shape[1]) for j in range(data.shape[2])]
    value_list = []
    values = data.values[0]

    for i in tqdm(index_list):
        neighbor = get_neighbors(values, *i)
        neighbor += [i[0], i[1]]
        value_list.append(neighbor)

    df_neighbors = pd.DataFrame(value_list, columns = ['v1', 'v2', 'v3','v4', 'v5', 'v6','v7', 'v8', 'v9', 'row', 'col'])   
    return df_neighbors

In [6]:
def add_row_and_col(df):
        # add row and column features:
        df['col'] = df.groupby("x").ngroup() # xshape 2663 
        df['row'] = df.groupby("y").ngroup(ascending=False) # yshape 1462
        return df

## Main

In [10]:
def main(mw_path, opt_path, out_path):
    # get files:
    mw_files = [f for f in listdir(mw_path) if isfile(join(mw_path, f))]
    opt_files = [f for f in listdir(opt_path) if isfile(join(opt_path, f))]

    # # TEMP
    # opt_files = [opt_files[-1]]   

    for opt_file in opt_files: # opt files have missing data so vice versa would give errors
        melt_date =  opt_file[:10]
        for mw_file in mw_files:
            if mw_file.startswith(melt_date):
                print(melt_date)
                data_opt = xarray.open_dataarray(opt_path + opt_file)
                data_mw = xarray.open_dataarray(mw_path + mw_file)
                #run_all_and_combine(data_mw, data_opt, melt_date)
                # to pandas:
                df_mw = raster_to_pandas(data_mw, 'mw')
                df_opt = raster_to_pandas(data_opt, 'opt')
                # merge the two dataframes:
                df_combined = pd.merge(df_mw, df_opt, how = 'left', on = ['y', 'x']) # left smaller mw, right - opt
                # add row and column features:
                df_combined = add_row_and_col(df_combined)
                # get neighbours:
                df_neighbors = get_neighbours_df(data_mw)
                # merge neighbours:
                df_comb = pd.merge(df_combined, df_neighbors, how = 'left', on = ['row', 'col'])
                # remove water in mw:
                df_comb = df_comb.loc[df_comb['mw_value'] != -1] # suppress warning?
                # write to parquet:
                df_comb.to_parquet(out_path + 'melt_'+ melt_date + '.parquet.gzip', index= False)
    return
                

In [11]:
mw_path = r"../Data/microwave-rs/mw_interpolated/"
opt_path = r"../Data/optical-rs/"

out_path = r"../Data/combined/to_pandas/"

main(mw_path, opt_path, out_path)

2019-06-08


100%|██████████| 3893306/3893306 [00:30<00:00, 127141.62it/s]


2019-06-10


100%|██████████| 3893306/3893306 [00:36<00:00, 107047.51it/s]


2019-06-11


...

100%|██████████| 3893306/3893306 [00:31<00:00, 123260.12it/s]


2019-07-30


100%|██████████| 3893306/3893306 [00:31<00:00, 124643.18it/s]


2019-07-31


100%|██████████| 3893306/3893306 [00:31<00:00, 122405.10it/s]
